In [1]:
import pandas as pd
import json
import pyomo.environ as pyo
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
with open("../data/mock.json") as f:
    d = json.load(f)
nodes, channels = pd.DataFrame(d["nodes"]), pd.DataFrame(d["edges"])

In [ ]:
#G = nx.DiGraph()
#
## Add nodes to the graph
#for _, node in nodes.iterrows():
#    G.add_node(node['pub_key'], alias=node['alias'])
#
## Add edges to the graph
#for _, edge in channels.iterrows():
#    G.add_edge(edge['node1_pub'], edge['node2_pub'], capacity=edge['capacity'], base_fee=edge['base_fee'], perc_fee=edge['perc_fee'])
#
## Draw the graph
#pos = nx.spring_layout(G)  # positions for all nodes
#nx.draw(G, pos, with_labels=True, node_size=2000, node_color='skyblue', font_size=10, font_color='black', font_weight='bold')
#
## Add edge labels
#edge_labels = {(u, v): f"{d['capacity']}" for u, v, d in G.edges(data=True)}
#nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
#
#plt.title('Network Graph')
#plt.show()

In [3]:
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
nodes.set_index("pub_key", inplace=True)

In [4]:
model = pyo.ConcreteModel()
model.NODES = pyo.Set(initialize=nodes.index)
model.CHANNELS = pyo.Set(initialize=channels.index, within=model.NODES*model.NODES)

In [ ]:
#model.add_component("addresses", pyo.Param(model.NODES, initialize=nodes["addresses"].to_dict(), within=pyo.Reals))
#model.add_component("capacity", pyo.Param(model.CHANNELS, initialize=channels["capacity"].to_dict(), within=pyo.Reals))
#model.add_component("base_fee", pyo.Param(model.CHANNELS, initialize=channels["base_fee"].to_dict(), within=pyo.Reals))
#model.add_component("perc_fee", pyo.Param(model.CHANNELS, initialize=channels["perc_fee"].to_dict(), within=pyo.Reals))

In [5]:
model.x = pyo.Var(model.CHANNELS, within=pyo.Binary)

In [6]:
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals)

$$\min \sum _{(i,j) \in E} (baseFee_{i,j} \times x_{i,j} + rateFee_{i,j} \times amount_{i,j})$$

In [7]:
def objective_function(model: pyo.ConcreteModel):
    return sum(channels.loc[k]["base_fee"] * model.x[k] for k in model.CHANNELS) + sum(channels.loc[i]["perc_fee"] * model.a[i] for i in model.CHANNELS)

In [8]:
model.totalCost = pyo.Objective(expr=objective_function(model), sense=pyo.minimize)

In [67]:
# Capacity constrain
def capacity_constraint_rule(model: pyo.ConcreteModel, a):
    return model.a[a] <= channels.loc[a]["capacity"] * model.x[a] 

In [66]:
model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint_rule, name="Capacity constraint")

(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
ERROR: Rule failed when generating expression for Constraint
CapacityConstraint with index ('node1', 'node2'): TypeError:
capacity_constraint_rule() takes 2 positional arguments but 3 were given
ERROR: Constructing component 'CapacityConstraint' from data=None failed:
        TypeError: capacity_constraint_rule() takes 2 positional arguments but
        3 were given


TypeError: capacity_constraint_rule() takes 2 positional arguments but 3 were given

In [62]:
# Flow constrain

model.a[("node2", "node3")].pprint()

{Member of a} : Size=25, Index=CHANNELS
    Key                : Lower : Value : Upper : Fixed : Stale : Domain
    ('node2', 'node3') :     0 :  None :  None : False :  True : NonNegativeReals
